# <center>R: Porównanie algorytmów</center>
---
## 1. Konstrukcja modeli i mierników parametrów
### 1.1 Biblioteki i edycja danych
#### 1.1.1 Biblioteki

In [1]:
library(dplyr)
library(data.table)
library(lme4)
library(nlme)
library(mgcv)
library(profmem)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

Loading required package: Matrix

Attaching package: 'nlme'

The following object is masked from 'package:lme4':

    lmList

The following object is masked from 'package:dplyr':

    collapse

This is mgcv 1.8-28. For overview type 'help("mgcv-package")'.


#### 1.1.2 Wczytanie i edycja danych

In [2]:
data_set <- fread("BIG_1.csv")
data_set$fac <- as.factor(data_set$fac)

head(data_set)

V1,y,x0,x1,x2,x3,f,f0,f1,f2,f3,fac
1,9.698274,0.28274594,0.5118063,0.6582616,0.4155267,10.60030,1.5519667,2.783231,3.265107,0,1
2,14.007447,0.12618791,0.3461578,0.2808795,0.4620316,16.98987,0.7722572,1.998337,8.219271,0,2
3,18.608061,0.21070526,0.9641912,0.7695708,0.9338649,18.78837,1.2293125,6.878374,1.680687,0,3
4,14.800216,0.08276877,0.4074184,0.3876047,0.5187097,19.40600,0.5142109,2.258807,4.632986,0,4
5,14.548328,0.75797857,0.7225473,0.3940279,0.9478225,13.05678,1.3783252,4.242254,4.436202,0,1
6,13.633780,0.61542552,0.4334147,0.6122287,0.9500592,13.50558,1.8699413,2.379355,3.256285,0,2


### 1.2 Funkcje
#### 1.2.1 Funkcja tworząca model

In [3]:
LMM <- function(model) {
  if(model == "lme4") {
    LMM_lme4 <- lmer(data=data_set, y ~ x0 + x1 + x2 + (1|fac))
    return(LMM_lme4)
  }
  else if(model == "nlme") {
    LMM_nlme <- lme(data=data_set, y ~ x0 + x1 + x2, random=~1|fac)
    return(LMM_nlme)
  }
  else if(model == "bam") {
    LMM_bam <- bam(data=data_set, y ~ x0 + x1 + x2 + s(fac, bs="re"))
    return(LMM_bam)
  }
}

#### 1.2.2 Funkcja sprawdzająca czas

In [4]:
check_time <- function(model, n) {
  times <- c()

  for(i in 1:n) {
    start_time <- Sys.time()
    LMM(model)
    end_time <- Sys.time()
    times <- c(times, round(end_time - start_time, 4))
    }
    return(times)
  }

#### 1.2.3 Funkcja mierząca ilość zużytego RAMu

In [5]:
check_RAM <- function(model) {
  prof_mem <- profmem({
    LMM(model)
      })
  sum_in_MB <- round(sum(prof_mem$bytes[!is.na(prof_mem$bytes)]) * 1e-6, 4)
  return(sum_in_MB)
}

## 2. Symulacje
### 2.1 Podsumowanie modelu
#### 2.1.1 "lme4"

In [6]:
 summary(LMM("lme4"))

Linear mixed model fit by REML ['lmerMod']
Formula: y ~ x0 + x1 + x2 + (1 | fac)
   Data: data_set

REML criterion at convergence: 509391.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4542 -0.6182  0.0196  0.6605  3.8117 

Random effects:
 Groups   Name        Variance Std.Dev.
 fac      (Intercept) 15.01    3.874   
 Residual              9.54    3.089   
Number of obs: 100000, groups:  fac, 4

Fixed effects:
            Estimate Std. Error  t value
(Intercept) 15.10717    1.93718    7.799
x0          -0.02337    0.03388   -0.690
x1           5.95760    0.03384  176.031
x2          -5.47141    0.03380 -161.864

Correlation of Fixed Effects:
   (Intr) x0     x1    
x0 -0.009              
x1 -0.009  0.004       
x2 -0.009  0.003  0.008

#### 2.1.2 "nlme"

In [7]:
summary(LMM("nlme"))

Linear mixed-effects model fit by REML
 Data: data_set 
       AIC      BIC    logLik
  509403.5 509460.6 -254695.8

Random effects:
 Formula: ~1 | fac
        (Intercept) Residual
StdDev:    3.873859 3.088759

Fixed effects: y ~ x0 + x1 + x2 
                Value Std.Error    DF    t-value p-value
(Intercept) 15.107168 1.9371783 99993    7.79854  0.0000
x0          -0.023373 0.0338783 99993   -0.68991  0.4902
x1           5.957597 0.0338439 99993  176.03148  0.0000
x2          -5.471413 0.0338025 99993 -161.86435  0.0000
 Correlation: 
   (Intr) x0     x1    
x0 -0.009              
x1 -0.009  0.004       
x2 -0.009  0.003  0.008

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-4.45415170 -0.61821428  0.01956955  0.66054780  3.81169918 

Number of Observations: 100000
Number of Groups: 4 

#### 2.1.3 "bam"

In [8]:
summary(LMM("bam"))


Family: gaussian 
Link function: identity 

Formula:
y ~ x0 + x1 + x2 + s(fac, bs = "re")

Parametric coefficients:
            Estimate Std. Error  t value Pr(>|t|)    
(Intercept) 15.10717    1.93717    7.799 6.32e-15 ***
x0          -0.02337    0.03388   -0.690     0.49    
x1           5.95760    0.03384  176.031  < 2e-16 ***
x2          -5.47141    0.03380 -161.864  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Approximate significance of smooth terms:
       edf Ref.df     F p-value    
s(fac)   3      3 39321  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

R-sq.(adj) =  0.639   Deviance explained = 63.9%
fREML = 2.547e+05  Scale est. = 9.5404    n = 100000

### 2.2 Sprawdzenie czasu konstrukcji modelu
#### 2.2.1 100 razy z użyciem funkcji "check_time(model, n)"

In [9]:
LMM_lme4_times <- check_time("lme4", 100)
LMM_nlme_times <- check_time("nlme", 100)
LMM_bam_times <- check_time("bam", 100)

data.frame(LMM_lme4_times, LMM_nlme_times, LMM_bam_times) %>%
    summarise(lme4_mean = round(mean(LMM_lme4_times), 4),
              lme4_sd = round(sd(LMM_lme4_times), 4),
              nlme_mean = round(mean(LMM_nlme_times), 4),
              nlme_sd = round(sd(LMM_nlme_times), 4),
              bam_mean = round(mean(LMM_bam_times), 4),
              bam_sd = round(sd(LMM_bam_times), 4))

lme4_mean,lme4_sd,nlme_mean,nlme_sd,bam_mean,bam_sd
0.8685,0.0812,1.1749,0.084,1.3588,0.1339


### 2.3 Sprawdzenie zużytej pamięci RAM
#### 2.3.1 Z użyciem funkcji "check_RAM(model)"

In [10]:
lme4_RAM <- check_RAM("lme4")
nlme_RAM <- check_RAM("nlme")
bam_RAM <- check_RAM("bam")

data.frame(lme4_RAM, nlme_RAM, bam_RAM)

lme4_RAM,nlme_RAM,bam_RAM
175.7481,174.6259,171.6016


#### 2.3.2 Z użyciem wbudowanego w R narzędzia profilowania

https://rpubs.com/kamilpytlak/LMM_BIG_1